<h1 style="font-size:50px;" align="center">Inferencia Bayesiana usando el Motor de Inferencia MIB</h1>

<h1 style="font-size:30px;" align="center">Quien escribió este cuento</h1>

In [8]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import glob
import nltk

nltk.download('punkt')
from nltk.tokenize import word_tokenize

from collections import Counter # regresa un diccionario con conteos
from collections import OrderedDict # diccionarios ordenados
from itertools import product,islice   # calcula producto cartesiano y saca una rebanada de diccionario


import mib as mb
import tpproc as tp

import math as mt

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rober\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
archivos = glob.glob('./Train/*/*')
archivos, nombres = tp.carga_cuentos(archivos)
df_train = tp.lee_cuentos(archivos, stw=True, remove_punkt=True)
df_train.head()


leyendo...
tamaño del contenido de archivos cargados:             9 KB


,titulo,tipo,autor,texto
0,catástrofes,minicuento,alfonso reyes,pasa hablado cierta catástrofe cósmica amenaza...
1,del perfecto gobernante,minicuento,alfonso reyes,entiende perfecto gobernante perfecto lleno pe...
2,ley profunda,minicuento,alfonso reyes,trata libertarnos simplemente enseñarnos descu...
3,los alivios,minicuento,alfonso reyes,nombres usan castellano restroom impropios bañ...
4,el buen rey león,fábula,esopo,león enojoso cruel violento sino tratable just...


In [10]:
archivos = glob.glob('./Test/*')
archivos, nombres = tp.carga_cuentos(archivos)
df_test = tp.lee_cuentos(archivos, test=True, stw=True, remove_punkt=True)
df_test.head()


leyendo...
tamaño del contenido de archivos cargados:             3 KB


,titulo,texto
0,divina fragilidad,dios creó universo sentía solo eternidad empez...
1,dos seres iguales,cierto día profeta sharía encontró niña jardín...
2,el caballo y el lobo,estación blandos céfiros hacen verdear campos ...
3,el cetro,dijo rey esposa señora verdaderamente reina de...
4,el impostor,cansada esperar hombre llegó puntual sonreírme...


## Formateo de Data Frames de trabajo (paso a paso)

In [11]:
df_train_T = df_train.copy()
df_test_T = df_test.copy()


In [12]:
from collections import Counter  # regresa un diccionario con conteos

df_train_T['palabras'] = df_train_T['texto'].apply(lambda x: x.split())
df_train_T['total'] = df_train_T['palabras'].apply(lambda x: len(x))
df_train_T['conteos'] = df_train_T['palabras'].apply(lambda x: Counter(x))

df_train_T = df_train_T.sort_values(by="total", ascending=False)
df_train_T_c = df_train_T.copy()
df_train_T.head()
# df_train_T.tail()


,titulo,tipo,autor,texto,palabras,total,conteos
7,chac mool,cuento,carlos fuentes,hace tiempo filiberto murió ahogado acapulco s...,"[hace, tiempo, filiberto, murió, ahogado, acap...",1807,"{'hace': 5, 'tiempo': 4, 'filiberto': 10, 'mur..."
27,nos han dado la tierra,cuento,juan rulfo,después tantas horas caminar encontrar sombra ...,"[después, tantas, horas, caminar, encontrar, s...",685,"{'después': 3, 'tantas': 1, 'horas': 2, 'camin..."
24,el almohadón de plumas,cuento,horacio quiroga,luna miel largo escalofrío rubia angelical tím...,"[luna, miel, largo, escalofrío, rubia, angelic...",653,"{'luna': 1, 'miel': 1, 'largo': 6, 'escalofrío..."
11,la luz es como el agua,cuento,gabriel garcía márquez,navidad niños volvieron pedir bote remos acuer...,"[navidad, niños, volvieron, pedir, bote, remos...",564,"{'navidad': 1, 'niños': 7, 'volvieron': 1, 'pe..."
29,desquite,cuento,josé saramago,muchacho venía río descalzo pantalones arreman...,"[muchacho, venía, río, descalzo, pantalones, a...",512,"{'muchacho': 14, 'venía': 1, 'río': 7, 'descal..."


In [13]:
df_train_T['conteos'] = df_train_T['conteos'].apply(
    lambda x: {k: v for k, v in sorted(x.items(), key=lambda item: item[1], reverse=True)})
df_train_T.head()


,titulo,tipo,autor,texto,palabras,total,conteos
7,chac mool,cuento,carlos fuentes,hace tiempo filiberto murió ahogado acapulco s...,"[hace, tiempo, filiberto, murió, ahogado, acap...",1807,"{'chac': 32, 'mool': 28, 'agua': 11, 'filibert..."
27,nos han dado la tierra,cuento,juan rulfo,después tantas horas caminar encontrar sombra ...,"[después, tantas, horas, caminar, encontrar, s...",685,"{'llano': 13, 'tierra': 12, 'dice': 9, 'aquí':..."
24,el almohadón de plumas,cuento,horacio quiroga,luna miel largo escalofrío rubia angelical tím...,"[luna, miel, largo, escalofrío, rubia, angelic...",653,"{'alicia': 17, 'jordán': 16, 'día': 8, 'cama':..."
11,la luz es como el agua,cuento,gabriel garcía márquez,navidad niños volvieron pedir bote remos acuer...,"[navidad, niños, volvieron, pedir, bote, remos...",564,"{'bote': 9, 'dijo': 9, 'niños': 7, 'papá': 7, ..."
29,desquite,cuento,josé saramago,muchacho venía río descalzo pantalones arreman...,"[muchacho, venía, río, descalzo, pantalones, a...",512,"{'muchacho': 14, 'río': 7, 'hacia': 6, 'quedó'..."


In [14]:
df_train_T['topw'] = df_train_T['conteos'].apply(lambda x: list(x.keys())[:5])
df_train_T.head()


,titulo,tipo,autor,texto,palabras,total,conteos,topw
7,chac mool,cuento,carlos fuentes,hace tiempo filiberto murió ahogado acapulco s...,"[hace, tiempo, filiberto, murió, ahogado, acap...",1807,"{'chac': 32, 'mool': 28, 'agua': 11, 'filibert...","[chac, mool, agua, filiberto, hoy]"
27,nos han dado la tierra,cuento,juan rulfo,después tantas horas caminar encontrar sombra ...,"[después, tantas, horas, caminar, encontrar, s...",685,"{'llano': 13, 'tierra': 12, 'dice': 9, 'aquí':...","[llano, tierra, dice, aquí, pueblo]"
24,el almohadón de plumas,cuento,horacio quiroga,luna miel largo escalofrío rubia angelical tím...,"[luna, miel, largo, escalofrío, rubia, angelic...",653,"{'alicia': 17, 'jordán': 16, 'día': 8, 'cama':...","[alicia, jordán, día, cama, largo]"
11,la luz es como el agua,cuento,gabriel garcía márquez,navidad niños volvieron pedir bote remos acuer...,"[navidad, niños, volvieron, pedir, bote, remos...",564,"{'bote': 9, 'dijo': 9, 'niños': 7, 'papá': 7, ...","[bote, dijo, niños, papá, padres]"
29,desquite,cuento,josé saramago,muchacho venía río descalzo pantalones arreman...,"[muchacho, venía, río, descalzo, pantalones, a...",512,"{'muchacho': 14, 'río': 7, 'hacia': 6, 'quedó'...","[muchacho, río, hacia, quedó, agua]"


In [15]:
top = 5
df_train_T['topw'] = df_train_T['conteos'].apply(lambda x: list(
    {k: v for k, v in sorted(x.items(), key=lambda item: item[1], reverse=True)}.keys())[:top])
df_train_T.head()


,titulo,tipo,autor,texto,palabras,total,conteos,topw
7,chac mool,cuento,carlos fuentes,hace tiempo filiberto murió ahogado acapulco s...,"[hace, tiempo, filiberto, murió, ahogado, acap...",1807,"{'chac': 32, 'mool': 28, 'agua': 11, 'filibert...","[chac, mool, agua, filiberto, hoy]"
27,nos han dado la tierra,cuento,juan rulfo,después tantas horas caminar encontrar sombra ...,"[después, tantas, horas, caminar, encontrar, s...",685,"{'llano': 13, 'tierra': 12, 'dice': 9, 'aquí':...","[llano, tierra, dice, aquí, pueblo]"
24,el almohadón de plumas,cuento,horacio quiroga,luna miel largo escalofrío rubia angelical tím...,"[luna, miel, largo, escalofrío, rubia, angelic...",653,"{'alicia': 17, 'jordán': 16, 'día': 8, 'cama':...","[alicia, jordán, día, cama, largo]"
11,la luz es como el agua,cuento,gabriel garcía márquez,navidad niños volvieron pedir bote remos acuer...,"[navidad, niños, volvieron, pedir, bote, remos...",564,"{'bote': 9, 'dijo': 9, 'niños': 7, 'papá': 7, ...","[bote, dijo, niños, papá, padres]"
29,desquite,cuento,josé saramago,muchacho venía río descalzo pantalones arreman...,"[muchacho, venía, río, descalzo, pantalones, a...",512,"{'muchacho': 14, 'río': 7, 'hacia': 6, 'quedó'...","[muchacho, río, hacia, quedó, agua]"


## Todo en 1
### Data Frames de Trabajo
- **Se puede jugar con el parámetro _top_ para aumentar o reducir el vocabulario**
- Se crean diccionarios para indexar las palabras a valores numéricos

In [16]:
df_train_T = df_train.copy()
df_test_T = df_test.copy()

"""
Cambiar si se desea (y se puede) aumentar el vocabulario
"""
top = 5

df_train_T['topw'] = df_train_T['texto'].apply(lambda x: [w[0] for w in sorted(
    Counter(x.split()).items(), key=lambda item: item[1], reverse=True)[:top]])
df_test_T['topw'] = df_test_T['texto'].apply(lambda x: [w[0] for w in sorted(
    Counter(x.split()).items(), key=lambda item: item[1], reverse=True)[:top]])

# Vocabulario de entrenamiento y prueba
vTR = {}
vTS = {}

# Obtenemos el vocabulario de entrenamiento
i = 0
for l in df_train_T.topw.values:
    for w in l:
        if w not in vTR:
            vTR[w] = i
            i += 1
vTR['UNK'] = i  # Agregamos palabra desconocida 'UNK'

print(len(vTR), 'palabras en vocabulario entrenamiento, incluyendo palabra desconocida (UNK)')

# Sustituimos palabra que no esta en vTR por palabra desconocida 'UNK' en textos de prueba
df_test_T['topw'] = df_test_T['topw'].apply(
    lambda x: [w if w in vTR.keys() else 'UNK' for w in x])

# Obtenemos el vocabulario de prueba
i = 0
for l in df_test_T.topw.values:
    for w in l:
        if w not in vTS:
            vTS[w] = i
            i += 1

print(len(vTS), 'palabras en vocabulario de prueba, incluyendo palabra desconocida (UNK)')

df_tr = df_train_T.filter(['titulo', 'tipo', 'autor'], axis=1)
df_ts = df_test_T.filter(['titulo'], axis=1)
for i in range(top):
    df_tr['w{}'.format(i)] = ''
for i in range(top):
    df_ts['w{}'.format(i)] = ''

for i, row in enumerate(df_train_T.iterrows()):
    l = row[1]['topw']
    for j, w in enumerate(l):
        df_tr.iloc[i, df_tr.columns.get_loc('w{}'.format(j))] = w
for i, row in enumerate(df_test_T.iterrows()):
    l = row[1]['topw']
    for j, w in enumerate(l):
        df_ts.iloc[i, df_ts.columns.get_loc('w{}'.format(j))] = w

display(df_tr.head())
display(df_ts.head())


136 palabras en vocabulario entrenamiento, incluyendo palabra desconocida (UNK)
19 palabras en vocabulario de prueba, incluyendo palabra desconocida (UNK)


,titulo,tipo,autor,w0,w1,w2,w3,w4
0,catástrofes,minicuento,alfonso reyes,catástrofe,parte,pasa,hablado,cierta
1,del perfecto gobernante,minicuento,alfonso reyes,gobernante,guerra,perfecto,paz,lleno
2,ley profunda,minicuento,alfonso reyes,vida,trata,libertarnos,simplemente,enseñarnos
3,los alivios,minicuento,alfonso reyes,alivios,nombres,usan,castellano,restroom
4,el buen rey león,fábula,esopo,liebre,justicia,león,enojoso,cruel


,titulo,w0,w1,w2,w3,w4
0,divina fragilidad,UNK,solo,UNK,UNK,UNK
1,dos seres iguales,UNK,dijo,UNK,UNK,UNK
2,el caballo y el lobo,lobo,dijo,UNK,UNK,UNK
3,el cetro,UNK,dijo,UNK,UNK,UNK
4,el impostor,UNK,dijo,UNK,hombre,UNK


## Preguntas

1. ¿Quién escribió "El caballo y el Lobo"?
2. "La escopeta" ¿es una fábula?
3. ¿Qué probabilidad hay de que "La muerte" haya sido escrita por José Saramago
4. ¿Pudo "La marcha del caracol" haber sido escrita por García Márquez y Fuentes en co-autoría?
5. Dado que "El águila de ala cortada y la zorra" es una fábula, ¿qué probabilidad hay de que haya sido escrita por Jalil Gibrán?
6. ¿Qué proporción hay de minicuentos?
7. ¿Qué cuentos son de Carlos Onetti?
8. ¿Quién dijo: "Desde que abrí los ojos me di cuenta que mi sitio no estaba aquí, donde yo estoy, sino en donde no estoy ni he estado nunca. En alguna parte hay un lugar vacío y ese vacío se llenará de mí y yo me asentaré en ese hueco que insensiblemente rebosará de mí, pleno de mí hasta volverse fuente o surtidor. Y mi vacío, el vacío de mí que soy ahora, se llenará de sí, pleno de sí, pleno de ser hasta los bordes."?
9. ¿Quién es el autor con más cuentos?
10. ¿Qué probablidad hay de quien haya escrito "Retiniblastoma" haya también escrito "El sol y las ranas"?

# Avance de una solución posible

## Modelo:
### Suponiendo independencia condicional de las palabras conociendo el tipo:

<img src="Cuentos-Modelo.jpeg" width=400/> <img src="Cuentos-Modelo-2.jpeg" width=400/>

### $P(A)P(T|A)\prod_{i=1}^N P(W_i|AT)$

## Variables

### Conteo de autores y tipos de cuento

In [17]:
# Ocurrencias de cada autor y conteos de ocurrencias y de número de autores
oc_autor = Counter(df_train_T.autor)
tot_oc_aut = np.sum(list(oc_autor.values()))
tot_autores = len(oc_autor)

print('{0} autores en total: {1}'.format(tot_autores, list(oc_autor.keys())))
print('{} menciones totales'.format(tot_oc_aut))

# Ocurrencias de cada tipo y conteos de ocurrencias y de número de tipos
oc_tipo = Counter(df_train_T.tipo)
tot_oc_tipo = np.sum(list(oc_tipo.values()))
tot_tipos = len(oc_tipo)

print('{0} tipos de cuento en total: {1}'.format(
    tot_tipos, list(oc_tipo.keys())))
print('{} menciones totales'.format(tot_oc_tipo))


10 autores en total: ['alfonso reyes', 'esopo', 'carlos fuentes', 'gabriel garcía márquez', 'gibrán jalil gibrán', 'jean de la fontaine', 'juan carlos onetti', 'horacio quiroga', 'juan rulfo', 'josé saramago']
30 menciones totales
3 tipos de cuento en total: ['minicuento', 'fábula', 'cuento']
30 menciones totales


### Definición de variables MIB

#### **Nota Importante**
* Por ahora, el motor de inferencia sólo acepta valores numéricos como valores posibles de una variable.
* Por lo tanto, vamos a crear un mapeo entre los nombres de los autores y un valor entero mediante un diccionario.
* También creamos un mapeo inverso, para poder recuperar las etiquetas a partir de los enteros.
* Haremos lo mismo con los tipos.

In [18]:
"""
Autor
"""
# diccionario autor->valor
autor_val = dict(zip(oc_autor.keys(), range(len(oc_autor.keys()))))
# diccionario valor->autor
val_autor = dict(zip(range(len(oc_autor.keys())), oc_autor.keys()))
print(autor_val)
print(val_autor)
A = mb.Var('A')
A.set_values(range(len(autor_val)))
print(A.print_var())
print()
"""
Tipo
"""
# diccionnario tipo -> valor
tipo_val = dict(zip(oc_tipo.keys(), range(len(oc_tipo.keys()))))

# diccionario valor -> tipo
val_tipo = dict(zip(range(len(oc_tipo.keys())), oc_tipo.keys()))
print(tipo_val)
print(val_tipo)
T = mb.Var('T')
T.set_values(range(len(val_tipo)))
print(T.print_var())
"""
Palabras
"""
# Se crean de manera similar a como se hizo para el filtro SPAM
# Palabras, presente:1 ausente:0

# diccionario valor -> palabra.
val_w = dict(zip(range(len(vTR)), vTR.keys()))

W = {}

for i, w in enumerate(vTR):
    W[i] = {w: mb.Var(name='W'+str(i), values=[0, 1])}

for i in range(len(W)):
    print(f'W[{i}]: {W[i]}')


{'alfonso reyes': 0, 'esopo': 1, 'carlos fuentes': 2, 'gabriel garcía márquez': 3, 'gibrán jalil gibrán': 4, 'jean de la fontaine': 5, 'juan carlos onetti': 6, 'horacio quiroga': 7, 'juan rulfo': 8, 'josé saramago': 9}
{0: 'alfonso reyes', 1: 'esopo', 2: 'carlos fuentes', 3: 'gabriel garcía márquez', 4: 'gibrán jalil gibrán', 5: 'jean de la fontaine', 6: 'juan carlos onetti', 7: 'horacio quiroga', 8: 'juan rulfo', 9: 'josé saramago'}
('A', 10, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

{'minicuento': 0, 'fábula': 1, 'cuento': 2}
{0: 'minicuento', 1: 'fábula', 2: 'cuento'}
('T', 3, [0, 1, 2])
W[0]: {'catástrofe': <mib.Var object at 0x0000022C162A8340>}
W[1]: {'parte': <mib.Var object at 0x0000022C162A84C0>}
W[2]: {'pasa': <mib.Var object at 0x0000022C162A8370>}
W[3]: {'hablado': <mib.Var object at 0x0000022C162A85E0>}
W[4]: {'cierta': <mib.Var object at 0x0000022C16214C70>}
W[5]: {'gobernante': <mib.Var object at 0x0000022C16214D30>}
W[6]: {'guerra': <mib.Var object at 0x0000022C16214EB0>}
W[7]: 

## Identificación de parámetros (estadísticas)

### $P(A)$


* $t_A =$ Total de textos del autor A.
* $t =$ Total de textos.

$P(A) = \frac{1 + t_A} {|A| + t}$

In [19]:
# data frame de autor usando df_train_T.
df_autor = df_train_T.groupby(['autor']).count().reset_index()
df_autor


,autor,titulo,tipo,texto,topw
0,alfonso reyes,4,4,4,4
1,carlos fuentes,2,2,2,2
2,esopo,3,3,3,3
3,gabriel garcía márquez,4,4,4,4
4,gibrán jalil gibrán,5,5,5,5
5,horacio quiroga,2,2,2,2
6,jean de la fontaine,4,4,4,4
7,josé saramago,2,2,2,2
8,juan carlos onetti,2,2,2,2
9,juan rulfo,2,2,2,2


In [20]:
# Conteo de textos de cada autor.
count_a = Counter(df_train_T.autor.values)

# Lista que guradara la distribución de A
tabla_dA = {}
cardA = len(oc_autor.keys())

# Corrección de Laplace P(A) = (1 + No. texto de autor) / (|A| + total textos).
# Sacar cada valor de count_a, para cada autor, que guarda el conteo de textos del autor.
for a in autor_val:
    tabla_dA[autor_val[a]] = (1 + count_a[a]) / (cardA + tot_oc_aut)

# Distribución de probabilidades
PA = mb.Distrib(name='P(A)',variable=[A],tabla=tabla_dA)
PA.to_Frame()

Creating DataFrame for:  P(A)


,P(A)
"(A, 0)",0.125
"(A, 1)",0.100
"(A, 2)",0.075
"(A, 3)",0.125
"(A, 4)",0.150
"(A, 5)",0.125
"(A, 6)",0.075
"(A, 7)",0.075
"(A, 8)",0.075
"(A, 9)",0.075


### $P(T|A)$

* $t_{AT} =$ Ocurrecnias de textos del tipo T en el autor A.
* $t_A =$ Total de textos del autor A.

$ P(T|A) = \frac{1 + t_{AT}}{|T| + t_A} $

In [21]:
df_train_Autor = df_train_T.groupby(['autor']).count().reset_index()
df_train_TA = df_train_T.groupby(['autor', 'tipo']).count().reset_index()
df_train_TA

,autor,tipo,titulo,texto,topw
0,alfonso reyes,minicuento,4,4,4
1,carlos fuentes,cuento,1,1,1
2,carlos fuentes,minicuento,1,1,1
3,esopo,fábula,3,3,3
4,gabriel garcía márquez,cuento,1,1,1
5,gabriel garcía márquez,minicuento,3,3,3
6,gibrán jalil gibrán,cuento,4,4,4
7,gibrán jalil gibrán,minicuento,1,1,1
8,horacio quiroga,cuento,2,2,2
9,jean de la fontaine,minicuento,4,4,4


In [22]:
# El total de textos cada autor A: count_a[A]
# Diccionario de tipo -> No. de ocurrencias de cada tipo por cada autor.
dict_txtT = {}

# Total de textos de un autor de cada tipo.
for a in set(df_train_T.autor):
    count_0 = 0
    count_1 = 0
    count_2 = 0
    dict_temp = {}
    for e in df_train_TA.values:
        if a == e[0]:
            if val_tipo[0] == e[1]:
                count_0 += e[2]
            if  val_tipo[1] == e[1]:
                count_1 += e[2]
            if  val_tipo[2] == e[1]:
                count_1 += e[2]
    dict_temp = {val_tipo[0]:count_0, val_tipo[1]:count_1, val_tipo[2]:count_2}
    
    dict_txtT[a] = dict_temp

In [23]:
# Tabla para la distribución de P(T|A)
dT_A = {}

for a in autor_val:
    dT_A_aux = {}
    for t in tipo_val:
        pat = (1 + dict_txtT[a][t]) / (3 + count_a[a])
        dT_A_aux[tipo_val[t]] = pat
    dT_A[autor_val[a]] = dT_A_aux

PT_A = mb.DistribCond(name='P(T|A)',var=T, indep=[A], tabla=dT_A)
PT_A.to_Frame()

,"(T, 0)","(T, 1)","(T, 2)"
"(A, 0)",0.714286,0.142857,0.142857
"(A, 1)",0.166667,0.666667,0.166667
"(A, 2)",0.400000,0.400000,0.200000
"(A, 3)",0.571429,0.285714,0.142857
"(A, 4)",0.250000,0.625000,0.125000
"(A, 5)",0.714286,0.142857,0.142857
"(A, 6)",0.200000,0.600000,0.200000
"(A, 7)",0.200000,0.600000,0.200000
"(A, 8)",0.200000,0.600000,0.200000
"(A, 9)",0.400000,0.400000,0.200000


### $\prod_{i=1}^N P(W_i|AT)$


$\frac{P(W_i)P(A|W_i)P(T|W_i)}{P(AT)}$ = 
$\frac{P(W_i)P(A|W_i)P(T|W_i)}{P(A)P(T|A)}$

* $ P(W_i) = \frac{1 +  No.W_i}{|W_i| + No.Palabras} $
* $ P(T|W_i) = \frac{1 + No.W_iQueApareceEnT}{|T| + No.W_i} $
* $ P(A|W_i) = \frac{1 + No.W_iQueApareceEnA}{|A| + No.W_i} $

In [24]:
# Calcular el total de palabras.
df_train_p = df_train_T_c.reset_index().get(["total"]).values

total_w = 0
for c in df_train_p:
    total_w += c[0]

In [25]:
# Obtener todas los conteos de las palabras, jutno con los datos de autor y el tipo del texto.
df_train_text = df_train_T_c.reset_index().get(["autor", "tipo", "conteos"]).values

# Diccionario para los conteo de las palabras.
dict_Wi_C = {}
# Diccionario para los conteos de las palabras dado el tipo.
dict_Wi_T = {}
# Diccionario para los conteos de las palabras dado el autor.
dict_Wi_A = {}

# Para cada palabra del diccionario de entrenamiento obtener los conteos.
for w in vTR:
    count_w = 0
    count_wt = [0, 0, 0]
    count_wa = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

    dict_wt = {}
    dict_wa = {}

    for e in df_train_text:
        # Conteo de palabras en todo el texto de Wi.
        # w esta en el diccionario de conteos se obtiene el valor.
        if w in e[2]:
            count_w += e[2][w]

            # Conteo de palabras por tipo.
            for t in tipo_val:
                if t == e[1]:
                    count_wt[tipo_val[t]] += count_w

            # Conteo de palabras por autor.
            for a in autor_val:
                if a == e[0]:
                    count_wa[autor_val[a]] += count_w

    # Crear un diccionario dado la llave T regrese el conteo de la palabra.
    for t in tipo_val:
        dict_wt[t] = count_wt[tipo_val[t]]
    # Crear un diccionario dado la llave A regrese el conteo de la palabra.
    for a in autor_val:
        dict_wa[a] = count_wa[autor_val[a]]
    # key: w -> valor: cantidad de veces que la palabra sale en el texto.
    dict_Wi_C[w] = count_w
    dict_Wi_T[w] = dict_wt
    dict_Wi_A[w] = dict_wa


In [26]:
PW_AT = {}

for i, w in enumerate(vTR):
    # Tabla para la distribución de P(W|AT)
    dWi_AT = []
    pw = 0
    ptw = 0
    paw = 0
    pa = 0
    pta = 0
    # Para cada combinación de tipo y autor
    for a in autor_val:
        for t in tipo_val:
            # P(W = 1)
            pw = (1 + dict_Wi_C[w]) / (2 + total_w)
            
            # P(T = t|W = 1)
            ptw = (1 + dict_Wi_T[w][t]) / (len(tipo_val) + dict_Wi_C[w])
            
            # P(A = a|W = 1)
            paw = (1 + dict_Wi_A[w][a]) / (len(autor_val) + dict_Wi_C[w])
            
            # P(A = a)
            pa = PA.get_P(('A', a))
            
            # P(T = t|A = a)
            pta = PT_A.get_P([('A', a)])[0][('T', t)]

            # P(W_i = 1|A = a, T = t) = P(W_i)P(A|W_i)P(T|W_i) / P(A)P(T|A)
            # P(W_i)P(A|W_i)P(T|W_i)
            N = pw * paw * ptw
            D = pa * pta
            p = N / D
            dWi_AT.append((1-p, p))

    # Guardar la distribución de la condicional de w en un diccionario.
    PW_AT[w] = mb.DistribCond(name='P(W'+str(i)+'|AT)',
                              var=W[i][w], indep=[A, T], tabla=dWi_AT)

NameError: name 'autores_set' is not defined

In [ ]:
PW_AT['catástrofe'].to_Frame()

,"(W0, 0)","(W0, 1)"
"(A, T, 0, 0)",0.998726,0.001274
"(A, T, 0, 1)",0.998407,0.001593
"(A, T, 0, 2)",0.998407,0.001593
"(A, T, 1, 0)",0.998293,0.001707
"(A, T, 1, 1)",0.999893,0.000107
"(A, T, 1, 2)",0.999573,0.000427
"(A, T, 2, 0)",0.999052,0.000948
"(A, T, 2, 1)",0.999763,0.000237
"(A, T, 2, 2)",0.999526,0.000474
"(A, T, 3, 0)",0.999602,0.000398


P(W|AT) = 

In [ ]:
for i, w in enumerate(vTR):
    
    for a in autores_set:
        for t in tipos_set:
            N = 1 + dict_Wi_T[w][val_tipo[t]]
            D = 2 + 

## Pregunta 1.

1. ¿Quién escribió "El caballo y el Lobo"?

Pregunta: $ P(A|W_k) = \frac{P(A,W_k)}  {P(W_k)}$

$P(A|W_k)$:
* $W_t =$ palabras del texto que estén en vTR
* $W_m =$ palabras de muestreo

$W_t \cap W_m = \emptyset$

$W_k = W_t \cup W_m$

* N = $\sum_{T,{W_i ... W_n} \ {'caballo', 'lobo'}} P(ATW_i^n)$
* D = $\sum_{T,{W_i ... W_n} \ {'caballo', 'lobo'}} P(ATW_1^n)$        

In [ ]:
df_test_P = df_test.copy()

df_test_P['palabras'] = df_test_P['texto'].apply(lambda x: x.split())
df_test_P['total'] = df_test_P['palabras'].apply(lambda x: len(x))
df_test_P


,titulo,texto,palabras,total
0,divina fragilidad,dios creó universo sentía solo eternidad empez...,"[dios, creó, universo, sentía, solo, eternidad...",174
1,dos seres iguales,cierto día profeta sharía encontró niña jardín...,"[cierto, día, profeta, sharía, encontró, niña,...",131
2,el caballo y el lobo,estación blandos céfiros hacen verdear campos ...,"[estación, blandos, céfiros, hacen, verdear, c...",111
3,el cetro,dijo rey esposa señora verdaderamente reina de...,"[dijo, rey, esposa, señora, verdaderamente, re...",62
4,el impostor,cansada esperar hombre llegó puntual sonreírme...,"[cansada, esperar, hombre, llegó, puntual, son...",169
5,el león y el pastor,yendo león montaña erró camino pasando lugar l...,"[yendo, león, montaña, erró, camino, pasando, ...",129
6,el sol y las ranas,ranas decidieron celebrar consejo asustadas so...,"[ranas, decidieron, celebrar, consejo, asustad...",60
7,el águila de ala cortada y la zorra,cierto día hombre capturó águila cortó alas so...,"[cierto, día, hombre, capturó, águila, cortó, ...",76
8,la escopeta,noche cerrada estiré brazo encender lámpara me...,"[noche, cerrada, estiré, brazo, encender, lámp...",161
9,la marcha del caracol,ibas contar algún día elizabeth caracol avanzó...,"[ibas, contar, algún, día, elizabeth, caracol,...",229


In [ ]:
df_ts_p1 = df_test_P.get(['titulo','palabras']).values

# Obtener las palabras de un texto con el titulo 
def w_titulo(titulo):
    for e in df_ts_p1:
        if titulo == e[0]:
            return e[1]
    return None

In [ ]:
import random as rand
# Muestreo para reducción de la marginalicación
# Obtener las palabras del texto que estan en el diccionario de palabnras de train
w_p1 = w_titulo('el caballo y el lobo')

total_w_p1 = set()
for w in w_p1:
    if w in vTR:
        total_w_p1.add(w)
total_w_p1.add('UNK')  
# Obtener las palabras de muestreo.
# Catidad de palabras de muestreo que no estén en el texto.
max_muestreo = 10
muestreo_i = 0
muestreo_w_p1 = set()
while (muestreo_i < max_muestreo):
    w = rand.sample(list(vTR), 1)
    if not(w in list(total_w_p1)):
        muestreo_w_p1.add(w[0])
        muestreo_i += 1

print("Palabras en el diccionario de train:",total_w_p1)
print("Palabras de muestreo:", muestreo_w_p1)

Palabras en el diccionario de train: {'vida', 'parte', 'UNK', 'cierto', 'dijo', 'lobo'}
Palabras de muestreo: {'solo', 'águila', 'entonces', 'dinero', 'cierto', 'mujer', 'fantasma', 'UNK', 'lobo', 'zeus'}


In [ ]:
# Combinaciones binarias de n bits
bit = [[0, 1] for i in range(max_muestreo)]
binC = list(product(*bit))

In [ ]:
N = []

for a in autor_val:
    # Obtener prpbabilidad de P(A)
    N_aux = 0
    
    pa = PA.get_P(('A', autor_val[a]))
    for t in tipo_val:
        pt_a_aux = PT_A.get_P([('A',autor_val[a])])
        pta = pt_a_aux[0][('T',tipo_val[t])]
        pwat = 1
        for n in range(len(binC)):
            for w in total_w_p1:
                pwat *= ma.log(PW_AT[w].get_P([('A','T',autor_val[a],tipo_val[t])])[0]['W'+str(vTR[w]),1])
            for i,w in enumerate(muestreo_w_p1):
                pwat *= ma.log(PW_AT[w].get_P([('A','T',autor_val[a],tipo_val[t])])[0]['W'+str(vTR[w]),binC[0][i]])
        N_aux += ma.log(pa) * ma.log(pta) * pwat
    N.append([a,N_aux])   
    
idx, max_value= max(N, key=lambda item: item[1])

idx

'alfonso reyes'

## Pregunta 2.

2. "La escopeta" ¿es una fábula?

$ P(T|'escopeta','W_1','W_2','W_3','W_4') = \frac{P(T,'escopeta','W_1','W_2','W_3','W_4')}{P('escopeta','W_1','W_2','W_3','W_4')}$

## Pregunta 3.

3. ¿Qué probabilidad hay de que "La muerte" haya sido escrita por José Saramago

$ P(A|'muerte','W_1','W_2','W_3','W_4') = \frac{P(A,'muerte','W_1','W_2','W_3','W_4')}{P('muert','W_1','W_2','W_3','W_4')}$

## Pregunta 4.

4. ¿Pudo "La marcha del caracol" haber sido escrita por García Márquez y Fuentes en co-autoría?

$ P(T|'marcha','caraclo','W_2','W_3','W_4') = \frac{P(T,'marcha','caracol','W_2','W_3','W_4')}{P('marcha','caracol','W_2','W_3','W_4')}$

## Preguta 5.

5. Dado que "El águila de ala cortada y la zorra" es una fábula, ¿qué probabilidad hay de que haya sido escrita por Jalil Gibrán?

$ P(A|T,'águila','ala','cortada','zorra','W_4') = \frac{P(A,T,'águila','ala','cortada','zorra',W_4)}{P(T,'águila','ala','cortada','zorra','W_4')}$

## Pregunta 6.

6. ¿Qué proporción hay de minicuentos?

$\frac{TotalMinucuentos}{TotalTextos}$

## Preuguta 7.

7. ¿Qué cuentos son de Carlos Onetti?

P(T|A)
* T = 'cuento'
* A = 'Carlos Onetti'

## Pregunta 8.

8. ¿Quién dijo: "Desde que abrí los ojos me di cuenta que mi sitio no estaba aquí, donde yo estoy, sino en donde no estoy ni he estado nunca. En alguna parte hay un lugar vacío y ese vacío se llenará de mí y yo me asentaré en ese hueco que insensiblemente rebosará de mí, pleno de mí hasta volverse fuente o surtidor. Y mi vacío, el vacío de mí que soy ahora, se llenará de sí, pleno de sí, pleno de ser hasta los bordes."?

$P(A|W_k) = \frac{P(A,W_k)}{P(W_k)}$

## Pregunta 9.

9. ¿Quién es el autor con más cuentos?

P(T|A)
* T = 'cuento'
* El A que tenga nmayor proporción es el que tiene más cuentos.

## Pregunra 10. 

10. ¿Qué probablidad hay de quien haya escrito "Retiniblastoma" haya también escrito "El sol y las ranas"?